In [1]:

import tensorflow as tf
import torch
from transformers import TFAutoModelForSequenceClassification, BertTokenizer
#import nltk
#from nltk.tokenize import word_tokenize
#nltk.download('punkt')
import pandas as pd 
import keras
from keras.optimizers import Adam
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

2024-07-10 23:01:49.670703: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-10 23:01:49.695528: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:10575] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-10 23:01:49.695552: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1442] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-10 23:01:49.710943: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-10 23:01:50.498654: W tensorflow/compiler/tf

In [2]:
tokenizer = BertTokenizer.from_pretrained("google-bert/bert-base-uncased")
model = TFAutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=5)

/home/rpierson/anaconda3/envs/secondenvi/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/home/rpierson/anaconda3/envs/secondenvi/lib/python3.11/site-packages/transformers/utils/generic.py:319: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
2024-07-10 23:01:55.866545: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1928] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 10532 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1080 Ti, pci bus id: 0000:04:00.0, compute capability: 6.1
2024-07-10 23:01:55.867758: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1928] Created device /job:localhost/replica:0/task:0/device:GPU:1 

In [3]:
label_encoder = LabelEncoder()

In [4]:
df = pd.read_csv(f'/home/rpierson/Topic_Files/topic_0.csv')
df['Priority'] = label_encoder.fit_transform(df['Priority'])

for x in range(len(df)):
    if pd.isna(df.iloc[x]["Combined_Text"]):
        df.at[x, "Combined_Text"] = " "
        
count = 0
for x in range(len(df)):
    if pd.isna(df.iloc[x]["Combined_Text"]):
        count += count
        
count

0

In [5]:
df.tail()

,Combined_Text,Priority
1025,add new versions of equinox.jetty; orbit bundl...,2
1026,Help files doesnt open User Assistance Build I...,2
1027,compile error in pde api module from the sourc...,2
1028,compile error with org.eclipse.help.base due t...,2
1029,upgrade to Orbit stable build S20080427194908 ...,2


In [6]:
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)

In [7]:
def tokenize_data(texts, labels, tokenizer, max_len):
    encodings = tokenizer(texts.tolist(), truncation=True, padding=True, max_length=max_len)
    return tf.data.Dataset.from_tensor_slices((
        dict(encodings),
        labels
    ))


In [8]:
train_dataset = tokenize_data(train_df.Combined_Text, train_df.Priority, tokenizer, max_len=128)
val_dataset = tokenize_data(val_df.Combined_Text, val_df.Priority, tokenizer, max_len=128)


In [9]:
train_dataset = train_dataset.shuffle(len(train_df)).batch(16)
val_dataset = val_dataset.batch(16)

In [24]:
optimizer = tf.keras.optimizers.Ftrl(learning_rate=0.01)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metrics = [tf.keras.metrics.SparseCategoricalAccuracy()]

In [25]:
if torch.cuda.is_available():
    print("CUDA is available. Number of GPUs:", torch.cuda.device_count())
    print("CUDA device name:", torch.cuda.get_device_name(0))
else:
    print("CUDA is not available.")
torch.cuda.set_device(torch.device("cuda:0"))

CUDA is available. Number of GPUs: 8
CUDA device name: NVIDIA GeForce GTX 1080 Ti


In [26]:
if torch.cuda.is_available():
        print(f"LOG: Torch allocated Memory: \
                {torch.cuda.memory_allocated() / 1024 ** 2:.2f} MB", flush=True)
        print(f"LOG: Torch cached Memory: \
                {torch.cuda.memory_reserved() / 1024 ** 2:.2f} MB", flush=True)

if tf.config.experimental.list_physical_devices('GPU'):
    zero_info = tf.config.experimental.get_memory_info('GPU:0')
    print(f"LOG: TensorFlow, 0: Current memory usage: \
            {zero_info['current'] / 1024 ** 2:.2f} MB", flush=True)
    print(f"LOG: TensorFlow, 0: Peak memory usage: \
            {zero_info['peak'] / 1024 ** 2:.2f} MB", flush=True)

LOG: Torch allocated Memory:                 0.00 MB
LOG: Torch cached Memory:                 0.00 MB
LOG: TensorFlow, 0: Current memory usage:             420.93 MB
LOG: TensorFlow, 0: Peak memory usage:             507.83 MB


In [27]:
# TensorFlow GPU configuration
#physical_devices = tf.config.experimental.list_physical_devices('GPU')
#if physical_devices:
#    try:
#        for device in physical_devices:
#            tf.config.experimental.set_memory_growth(device, True)
#        print(f"LOG: TensorFlow GPU devices: {physical_devices}.", flush=True)
#    except RuntimeError as exception:
#        print(f"LOG: TensorFlow GPU devices: {physical_devices}.", flush=True)
#        print(f"LOG: TensorFlow GPU configuration error: {exception}", flush=True)
#else:
#    print("ERROR: No TensorFlow GPU devices found.", flush=True)

In [28]:
strategy = tf.distribute.MirroredStrategy()

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1', '/job:localhost/replica:0/task:0/device:GPU:2', '/job:localhost/replica:0/task:0/device:GPU:3', '/job:localhost/replica:0/task:0/device:GPU:4', '/job:localhost/replica:0/task:0/device:GPU:5', '/job:localhost/replica:0/task:0/device:GPU:6', '/job:localhost/replica:0/task:0/device:GPU:7')


In [32]:
if torch.cuda.is_available():
        print(f"LOG: Torch allocated Memory: \
                {torch.cuda.memory_allocated() / 1024 ** 2:.2f} MB", flush=True)
        print(f"LOG: Torch cached Memory: \
                {torch.cuda.memory_reserved() / 1024 ** 2:.2f} MB", flush=True)

if tf.config.experimental.list_physical_devices('GPU'):
    zero_info = tf.config.experimental.get_memory_info('GPU:0')
    print(f"LOG: TensorFlow, 0: Current memory usage: \
            {zero_info['current'] / 1024 ** 2:.2f} MB", flush=True)
    print(f"LOG: TensorFlow, 0: Peak memory usage: \
            {zero_info['peak'] / 1024 ** 2:.2f} MB", flush=True)

from tensorflow.keras.optimizers import Adam
# model.compile(optimizer=Adam(3e-5))  # No loss argument!

model.compile(optimizer='Ftrl', loss='sparse_categorical_crossentropy', metrics = ['Accuracy', 'Precision', 'Recall'])


LOG: Torch allocated Memory:                 0.00 MB
LOG: Torch cached Memory:                 0.00 MB
LOG: TensorFlow, 0: Current memory usage:             420.93 MB
LOG: TensorFlow, 0: Peak memory usage:             507.83 MB


In [34]:
model.add(layers.Dense(5, activation='softmax'))

AttributeError: 'TFBertForSequenceClassification' object has no attribute 'add'

In [33]:
model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=15
)

Epoch 1/15
Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


ValueError: in user code:

    File "/home/rpierson/anaconda3/envs/secondenvi/lib/python3.11/site-packages/tf_keras/src/engine/training.py", line 1398, in train_function  *
        return step_function(self, iterator)
    File "/home/rpierson/anaconda3/envs/secondenvi/lib/python3.11/site-packages/tf_keras/src/engine/training.py", line 1370, in run_step  *
        outputs = model.train_step(data)
    File "/home/rpierson/anaconda3/envs/secondenvi/lib/python3.11/site-packages/transformers/modeling_tf_utils.py", line 1643, in train_step  *
        self.compiled_metrics.update_state(y, y_pred, sample_weight)
    File "/home/rpierson/anaconda3/envs/secondenvi/lib/python3.11/site-packages/tf_keras/src/engine/compile_utils.py", line 620, in update_state  *
        metric_obj.update_state(y_t, y_p, sample_weight=mask)
    File "/home/rpierson/anaconda3/envs/secondenvi/lib/python3.11/site-packages/tf_keras/src/metrics/base_metric.py", line 153, in decorated  *
        result = update_state_fn(*

In [ ]:
print(keras.__version__)